In [3]:
!nvidia-smi

Mon Aug 29 10:21:00 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.08    Driver Version: 512.96       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| 95%   57C    P2   332W / 333W |   6504MiB / 24576MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA RTX A5000    On   | 00000000:05:00.0 Off |                  Off |
| 82%   

In [ ]:
%%time

from datasets import load_dataset, Features, ClassLabel

# data_files={"train": "./data/json/unsw_corpus_*.json.gz", "test": "./data/json/unsw_corpus_2.csv"}
# data_files={"./data/json/unsw_corpus_7.json"}
# data_features = Features({'text': datasets.Value('string')})
# data_dir="./data/json"

# class_label=Features({'label': ClassLabel(num_classes=6, names=['Hub', 'Energy Management','Health Monitor','Camera','Appliances','Non IoT'])})
datasets = load_dataset('json', data_dir="./data/json",split='train')#,features=class_label)

In [ ]:
datasets.unique('label')

In [ ]:
label_encoder = {'Hub':0, 'Energy Management':1,'Health Monitor':2,'Camera':3,'Appliances':4,'Non IoT':5,None:5}
def labelr(example):
    example['label'] = label_encoder[example['label']]
    return example

In [ ]:
datasets=datasets.map(labelr)

In [ ]:
datasets.features

In [ ]:
from datasets import load_dataset, ClassLabel, Value

new_features = datasets.features.copy()
new_features = datasets.features.copy()
new_features['label'] = ClassLabel(names=['Hub', 'Energy Management','Health Monitor','Camera','Appliances','Non IoT'])
new_features['text'] = Value('string')
datasets = datasets.cast(new_features)
datasets.features

In [ ]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./models", max_len=576)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = datasets.map(tokenize_function, batched=True)


In [ ]:
tokenized_datasets.features

In [ ]:
# tokenized_datasets.save_to_disk("./data/tokenized_dataset_category")


### load saved dataset from disk

In [2]:
from datasets import load_from_disk
tokenized_datasets = load_from_disk("./data/tokenized_dataset_category")

In [1]:
tokenized_datasets['label'][5000]

NameError: name 'tokenized_datasets' is not defined

In [7]:
tokenized_datasets = tokenized_datasets.remove_columns(["text"])

In [8]:
datasets = tokenized_datasets.train_test_split(test_size=0.05,stratify_by_column="label",seed=42 )

Loading cached split indices for dataset at data/tokenized_dataset_category/cache-b6ad6190e9aca3ee.arrow and data/tokenized_dataset_category/cache-89f06e6303712af5.arrow


In [9]:
d_train = datasets['test'].train_test_split(test_size=0.2,stratify_by_column='label',seed=42)

In [10]:
d_test = d_train['test'].train_test_split(test_size=0.5,stratify_by_column='label',seed=42)

In [11]:
from datasets.dataset_dict import DatasetDict
dd = DatasetDict({"train":d_train['train'],"test":d_test['train'],"validation":d_test['test']})

In [12]:
dd

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 116124
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 14516
    })
    validation: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 14516
    })
})

In [13]:
from transformers import RobertaForSequenceClassification

model = RobertaForSequenceClassification.from_pretrained("./models/checkpoint-10000", num_labels=6)

Some weights of the model checkpoint at ./models/checkpoint-10000 were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ./models/checkpoint-10000 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_p

In [14]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(52000, 768, padding_idx=1)
      (position_embeddings): Embedding(578, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN

In [15]:
# import numpy as np
# from datasets import load_metric

# metric = load_metric("accuracy")
# def compute_metrics(eval_pred):
#     logits, labels = eval_pred
#     predictions = np.argmax(logits, axis=-1)
#     return metric.compute(predictions=predictions, references=labels)

In [16]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="cat_trainer_3",
    learning_rate=1e-5,
    evaluation_strategy="epoch",    
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    load_best_model_at_end=True,
    save_strategy="epoch",
    gradient_accumulation_steps=8,
    seed=42

)

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dd['train'],
    eval_dataset=dd['validation']
#     compute_metrics=compute_metrics,
)

In [18]:
trainer.train()

/usr/local/lib/python3.6/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 116124
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 8
  Total optimization steps = 9060
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: fdaus. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.6/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
0,No log,0.404419
1,0.537600,0.379482
2,0.393500,0.369259
3,0.376800,0.367921
4,0.364100,0.362929
5,0.359300,0.372858
6,0.351400,0.362923
7,0.346200,0.363783
8,0.342800,0.365701
9,0.338800,0.363152


***** Running Evaluation *****
  Num examples = 14516
  Batch size = 16
Saving model checkpoint to cat_trainer_3/checkpoint-453
Configuration saved in cat_trainer_3/checkpoint-453/config.json
Model weights saved in cat_trainer_3/checkpoint-453/pytorch_model.bin
/usr/local/lib/python3.6/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 14516
  Batch size = 16
Saving model checkpoint to cat_trainer_3/checkpoint-906
Configuration saved in cat_trainer_3/checkpoint-906/config.json
Model weights saved in cat_trainer_3/checkpoint-906/pytorch_model.bin
/usr/local/lib/python3.6/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vect

Configuration saved in cat_trainer_3/checkpoint-7248/config.json
Model weights saved in cat_trainer_3/checkpoint-7248/pytorch_model.bin
/usr/local/lib/python3.6/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 14516
  Batch size = 16
Saving model checkpoint to cat_trainer_3/checkpoint-7701
Configuration saved in cat_trainer_3/checkpoint-7701/config.json
Model weights saved in cat_trainer_3/checkpoint-7701/pytorch_model.bin
/usr/local/lib/python3.6/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
***** Running Evaluation *****
  Num examples = 14516

TrainOutput(global_step=9060, training_loss=0.35295400535297183, metrics={'train_runtime': 68452.0385, 'train_samples_per_second': 33.929, 'train_steps_per_second': 0.132, 'total_flos': 3.4611093433351066e+17, 'train_loss': 0.35295400535297183, 'epoch': 20.0})

In [19]:
# from huggingface_hub import notebook_login

# notebook_login()

In [20]:
# # Define a new Trainer with all the objects we constructed so far
# from transformers import TrainingArguments, Trainer

# repo_name = "finetuning-sentiment-model-3000-samples"

# training_args = TrainingArguments(
#     output_dir=repo_name,
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=16,
#     num_train_epochs=2,
#     weight_decay=0.01,
#     save_steps=500, 
#     push_to_hub=True,
#     report_to = 'wandb',
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_train,
#     eval_dataset=tokenized_test,
#     tokenizer=tokenizer,
#     data_collator=data_collator,
#     compute_metrics=compute_metrics,
# )

In [21]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 14516
  Batch size = 16
/usr/local/lib/python3.6/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.36292335391044617,
 'eval_runtime': 130.2335,
 'eval_samples_per_second': 111.461,
 'eval_steps_per_second': 3.486,
 'epoch': 20.0}

In [23]:
pred = trainer.predict(dd['test'])

***** Running Prediction *****
  Num examples = 14516
  Batch size = 16


In [53]:
from datasets import load_metric
import numpy as np

metric = load_metric("accuracy")

def ypred_ytrue(eval_pred):
    logits = eval_pred.predictions
    labels = eval_pred.label_ids
    predictions = np.argmax(logits, axis=-1)
    return predictions, labels

def compute_metrics(eval_pred):
    predictions, labels = ypred_ytrue(eval_pred)
    return metric.compute(predictions=predictions, references=labels)

In [54]:
compute_metrics(pred)

{'accuracy': 0.8140672361532103}

In [44]:
# pred.label_ids
unique, counts = np.unique(pred.label_ids, return_counts=True)
dict(zip(unique, counts))

{0: 1370, 1: 4904, 2: 231, 3: 3981, 4: 1665, 5: 2365}

In [81]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import multilabel_confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report

def evaluation_metric(pred):
    y_pred, y_true = ypred_ytrue(pred)

    confusion = confusion_matrix(y_true, y_pred)
    print('Confusion Matrix\n')
    print(confusion)

    #one-vs-rest way confusion matrix
    print('\nMultilabel Confusion Matrix\n')
    print(multilabel_confusion_matrix(y_true, y_pred,labels=[0, 1, 2,3,4,5]))
    
    #accuracy_score, precision_score, recall_score, f1_score
#     print('\nAccuracy: {:.2f}\n'.format(accuracy_score(y_true, y_pred)))

#     print('Micro Precision: {:.2f}'.format(precision_score(y_true, y_pred, average='micro')))
#     print('Micro Recall: {:.2f}'.format(recall_score(y_true, y_pred, average='micro')))
#     print('Micro F1-score: {:.2f}\n'.format(f1_score(y_true, y_pred, average='micro')))

#     print('Macro Precision: {:.2f}'.format(precision_score(y_true, y_pred, average='macro')))
#     print('Macro Recall: {:.2f}'.format(recall_score(y_true, y_pred, average='macro')))
#     print('Macro F1-score: {:.2f}\n'.format(f1_score(y_true, y_pred, average='macro')))

#     print('Weighted Precision: {:.2f}'.format(precision_score(y_true, y_pred, average='weighted')))
#     print('Weighted Recall: {:.2f}'.format(recall_score(y_true, y_pred, average='weighted')))
#     print('Weighted F1-score: {:.2f}'.format(f1_score(y_true, y_pred, average='weighted')))

    print('\nClassification Report\n')
    print(classification_report(y_true, y_pred, target_names=['Hub', 'Energy Management','Health Monitor','Camera','Appliances','Non IoT']))

In [82]:
evaluation_metric(pred)


Confusion Matrix

[[1358    7    1    3    1    0]
 [   6 3645    1  378  872    2]
 [   0    0  228    0    1    2]
 [   9  161    0 3038  717   56]
 [   1   18    1  356 1278   11]
 [   4    5    1   44   41 2270]]

Multilabel Confusion Matrix

[[[13126    20]
  [   12  1358]]

 [[ 9421   191]
  [ 1259  3645]]

 [[14281     4]
  [    3   228]]

 [[ 9754   781]
  [  943  3038]]

 [[11219  1632]
  [  387  1278]]

 [[12080    71]
  [   95  2270]]]

Classification Report

                   precision    recall  f1-score   support

              Hub       0.99      0.99      0.99      1370
Energy Management       0.95      0.74      0.83      4904
   Health Monitor       0.98      0.99      0.98       231
           Camera       0.80      0.76      0.78      3981
       Appliances       0.44      0.77      0.56      1665
          Non IoT       0.97      0.96      0.96      2365

         accuracy                           0.81     14516
        macro avg       0.85      0.87      0.85   